In [42]:

import tensorflow as tf
import numpy as np
import os
#from PhasedLSTMCell_v1 import *
from PhasedLSTMCell import *
import time
import sys
import pickle
from rnn_dynamic import *
import pandas as pd

# Load test data¶

In [2]:
max_interactions = 100

with open("pickles/movielens/X_test_" + str(max_interactions) + ".pickle", 'rb') as handle:
    X_test = pickle.load(handle)
with open("pickles/movielens/Y_test_" + str(max_interactions) + ".pickle", 'rb') as handle:
    Y_test = pickle.load(handle)

# Load Tensorflow model

In [3]:


# tensorflow model
checkpoint_path = 'checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000'

model_parameters = {}
model_parameters['opt'] = 'adam'
model_parameters['learning_rate'] = 0.001
model_parameters['n_hidden'] = 50
model_parameters['batch_size'] = 128
model_parameters['rnn_type'] = 'lstm2'
model_parameters['rnn_layers'] = 1
model_parameters['dropout'] = 0.2
model_parameters['l2_reg'] = 0.0
model_parameters['type_output'] = 'softmax'
model_parameters['max_steps'] = 3000000
model_parameters['padding'] = 'right'
model_parameters['n_input'] = X_test[0].toarray().shape[1]
model_parameters['n_output'] = Y_test[0].toarray().shape[1]
model_parameters['seq_length'] = X_test[0].toarray().shape[0]

#Create tensorflow model and train
print('Create model...')
model = RNN_dynamic(model_parameters)
model.create_model()


Create model...


C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# Evaluation

In [4]:


def evaluate_sample(predictions, y_true, k):
    idx_predictions = np.arange(len(predictions))
    sorted_pred, sorted_idx = zip(*sorted(zip(predictions,idx_predictions), reverse=True))
    # Recall
    _, y_true_idx = np.where(y_true ==1)
    correct_idx = set(sorted_idx[:k]).intersection(set(y_true_idx))
    num_pos_k = len(correct_idx)
    total_pos = len(y_true)
    recall_user_k = (num_pos_k/float(total_pos))
    # Sps
    first_movie = y_true_idx[0]
    if first_movie in sorted_idx[0:k]:
        sps_k = 1
    else:
        sps_k = 0
    # Map
    sum_precisions = 0
    actual_pos = 0
    for i in range(k):
        if sorted_idx[i] in y_true_idx:
            actual_pos += 1
        sum_precisions += actual_pos/float(i+1)
    ap_k = sum_precisions/min(k, len(y_true))
    
    return recall_user_k, sps_k, ap_k, num_pos_k, total_pos



In [5]:
# Test function

predictions = np.array([0.2, 0.75, 0.05])
y_true = np.array([[0, 1, 0],[0,0,1]])
k = 2
recall_user_k, sps_k, ap_k, num_pos_k, total_pos = evaluate_sample(predictions, y_true, k)
print('recall: ' + str(recall_user_k))
print('sps_k: ' + str(sps_k))
print('ap_k: ' + str(ap_k))
print('num_pos_k: ' + str(num_pos_k))
print('--------------')
predictions = np.array([0.1, 0.2, 0.55, 0.15])
y_true = np.array([[1,0,0,0], [0,0,0,1], [0,0,1,0]])
k = 3
recall_user_k, sps_k, ap_k, num_pos_k, total_pos = evaluate_sample(predictions, y_true, k)
print('recall: ' + str(recall_user_k))
print('sps_k: ' + str(sps_k))
print('ap_k: ' + str(ap_k))
print('num_pos_k: ' + str(num_pos_k))

recall: 0.5
sps_k: 1
ap_k: 0.75
num_pos_k: 1
--------------
recall: 0.6666666666666666
sps_k: 0
ap_k: 0.7222222222222222
num_pos_k: 2


In [9]:
# Make predictions in chunks

k = 10
recalls = []
spss = []
aps = []
num_poss = []
total_poss = []

num_movies = Y_test[0].toarray().shape[1]
batch_size = 100
for i in range(0, len(X_test), batch_size):
    x_test = [x.toarray() for x in X_test[i:i+batch_size]]
    y_test = [y.toarray() for y in Y_test[i:i+batch_size]]
    y_pred = model.predict(x_test, checkpoint_path)
    for j in range(len(y_pred)):
        recall_user_k, sps_k, ap_k, num_pos_k, total_pos = evaluate_sample(y_pred[j], y_test[j], k)
        recalls.append(recall_user_k)
        spss.append(sps_k)
        aps.append(ap_k)
        num_poss.append(num_pos_k)
        total_poss.append(total_pos)
    print(str(i) + '/' + str(len(X_test)))

print('Mean recall users: ' + str(np.mean(recalls)))
print('Mean spss: ' + str(np.mean(spss)))
print('MAP: ' + str(np.mean(aps)))
total_recall = np.sum(num_poss)/float(np.sum(total_poss))
print('Total Recall (no mean recall users): ' + str(total_recall))

load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
0/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
100/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
200/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
300/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
400/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
500/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
600/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/best_model/model_best.ckpt-128000
700/2232
load model: checkpoints/rep0-lstm2-50-1-128-adam-6000000-20170417-204007/b

# Look particular prediction

In [ ]:
# First let's load data about the movies
with open("../src/pickles/movielens/user_test_" + str(max_interactions) + ".pickle", 'rb') as handle:
    user_test = pickle.load(handle)
with open("../data/Movielens/ml-20m/df_date_2010.pickle", 'rb') as handle:
    df_date = pickle.load(handle)
dataset_file = "../data/Movielens/ml-20m/movies.csv"
df_movies = pd.read_csv(dataset_file)

# Build array mapping movie_id --> position in one-hot encoding
moviesData = {}
movieIds = np.zeros(max(df_date['movieId'].unique()) + 1, np.uint16)
i = 0
for movieId in df_date['movieId'].unique():
    movieIds[movieId] = i
    moviesData[i] = {}
    moviesData[i]['title'] = df_movies[df_movies['movieId'] == movieId].title
    moviesData[i]['genres'] = df_movies[df_movies['movieId'] == movieId].genres
    i += 1
    
    

In [16]:
# Look user with sps = 1
a = np.where(np.array(spss) == 1)

In [19]:
spss[2215]

1

In [68]:
# Look which was the first movie for this user
b = np.where(y_test[15][0] == 1)
b

(array([130], dtype=int64),)

In [27]:
# Look the probability for this movie
y_pred[15][130]

0.0097142299637198448

In [71]:
# Sort the predicted movies by probabilities
idx_predictions = np.arange(len(y_pred[15]))
sorted_pred, sorted_idx = zip(*sorted(zip(y_pred[15],idx_predictions), reverse=True))
print(sorted_pred[0:k])
print(sorted_idx[0:k])

(0.0097142299637198448, 0.008466687984764576, 0.0075912931933999062, 0.0069626700133085251, 0.0067433267831802368, 0.006249699741601944, 0.0061829485930502415, 0.0058726221323013306, 0.0058480473235249519, 0.0055826134048402309)
(130, 445, 8, 3, 309, 611, 73, 0, 53, 121)


In [72]:
# Let's see where where the movies previous to prediction time
c,d = np.where(x_test[15] == 1)
d

array([ 121,   99,    3,  309,  145,  147,  838,  445,   70,   78,  550,
        311,   90,  611,  533,   92,  478,  677,  269,  505,  155,  214,
        763,  391,  128,  668,  339,  245,  493,   81,    4,  952,   24,
        549,  138,  686,  374,  335,   75, 1335,   93,  519,  671, 1014,
        368,  229,  990,  112,  265,    0,  547,  142,  148,  430,  143,
        141,  174,  548,  186,  170,  913,  631,  189,  280,  315,  529,
         65,   76,   42,  118,  552,   73,  401,  927], dtype=int64)

In [77]:
# Predicted movie and real first movie
moviesData[130]

{'genres': 2873    Action|Crime|Drama|Thriller
 Name: genres, dtype: object, 'title': 2873    Fight Club (1999)
 Name: title, dtype: object}

In [75]:
# movies before prediction time
moviesData[73]

{'genres': 587    Crime|Horror|Thriller
 Name: genres, dtype: object, 'title': 587    Silence of the Lambs, The (1991)
 Name: title, dtype: object}

In [78]:
# the user data in df_date
df_date[df_date.userId == user_test[2215]].sort_values('date')

,userId,movieId,rating,timestamp,date
19837745,137322,2571,2.0,1420029674,2014-12-31 12:41:14
19837737,137322,1291,4.0,1420029677,2014-12-31 12:41:17
19837773,137322,7153,4.0,1420029681,2014-12-31 12:41:21
19837786,137322,58559,3.5,1420029690,2014-12-31 12:41:30
19837760,137322,4993,4.0,1420029693,2014-12-31 12:41:33
19837765,137322,5952,4.0,1420029694,2014-12-31 12:41:34
19837720,137322,293,5.0,1420029703,2014-12-31 12:41:43
19837794,137322,79132,4.0,1420029705,2014-12-31 12:41:45
19837726,137322,589,1.0,1420029715,2014-12-31 12:41:55
19837731,137322,1036,4.0,1420029720,2014-12-31 12:42:00


In [79]:
df_movies[df_movies.movieId == 2959]

,movieId,title,genres
2873,2959,Fight Club (1999),Action|Crime|Drama|Thriller


In [85]:
# Now a bad predicted example
idx = 2218
print(recalls[idx])
print(spss[idx])
print(aps[idx])
print(num_poss[idx])
print(total_poss[idx])

0.0
0
0.0
0
81


In [90]:
# Look which was the first movie for this user
b = np.where(Y_test[2218].toarray()[0] == 1)
b

(array([1612], dtype=int64),)

In [93]:
# Look the probability for this movie
y_pred[18][1612]

0.0018392826896160841

In [97]:
# Sort the predicted movies by probabilities
idx_predictions = np.arange(len(y_pred[18]))
sorted_pred, sorted_idx = zip(*sorted(zip(y_pred[18],idx_predictions), reverse=True))
print(sorted_pred[0:k])
print(sorted_idx[0:k])

(0.0030854102224111557, 0.0027906710747629404, 0.0027042878791689873, 0.0025688330642879009, 0.0024558035656809807, 0.002337039215490222, 0.0021375708747655153, 0.002118848729878664, 0.0019896230660378933, 0.0019626379944384098)
(309, 367, 368, 735, 54, 649, 7, 351, 633, 302)


In [98]:
# Let's see where where the movies previous to prediction time
c,d = np.where(x_test[18] == 1)
d

array([  551,  2016,   355,  1726,  3468,   203,  1059,   368,   370,
         367,  4395,  2461,  1453,  1576,  4902,  4812,   369,  2018,
        1484,   182,   532,  5099,  2462,  1010,  2029,  6971,  1016,
        6289,  1009,  4155,   339,   346,  2786,  2465,  1579,  1434,
         350,  1432,   372,   127,  3112,   734,   185,  3596,   332,
          41,   373,  1745,  1032,  1030,   554,  1037,   237,  1802,
         376,  4165,   284,   374,  4882,  5893,  7045,  8597,  2244,
         382,  1393,   506,  6895,  6183,   361,   343,  1793,  2246,
         947,   218,  3213,  2569,  2789,  1039,  2471,  2286,   496,
         610,   602,  1812,  1442,  2221,   914,  1055,  8723, 10374,
          48,  1069,   518,  1526,  4238,  3855,   916,  5606,  4154,  1351], dtype=int64)

In [99]:
# real first movie
moviesData[1612]

{'genres': 7142    Drama|Sci-Fi|Thriller
 Name: genres, dtype: object, 'title': 7142    The Butterfly Effect (2004)
 Name: title, dtype: object}

In [100]:
# real predicted movie
moviesData[309]

{'genres': 12525    Action|Crime|Drama|IMAX
 Name: genres, dtype: object, 'title': 12525    Dark Knight, The (2008)
 Name: title, dtype: object}

In [101]:
# last rated movie
moviesData[1351]

{'genres': 2514    Comedy
 Name: genres, dtype: object, 'title': 2514    Election (1999)
 Name: title, dtype: object}